<a href="https://colab.research.google.com/github/2stndard/polars/blob/main/7%EC%9E%A5_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
## 사전 실행 코드
import polars as pl
df_boxoffice = pl.read_csv("./movie/KOBIS_기간별박스오피스.csv", skip_rows = 4, try_parse_dates=True)
boxoffice_cols = df_boxoffice.columns
boxoffice_cols[4] = '매출액점유율'
df_boxoffice.columns = [i.replace(' ', '') for i in boxoffice_cols]
df_boxoffice = df_boxoffice.with_columns(
    pl.col('매출액', '누적매출액').str.replace_all(',', '').cast(pl.Int64),
    pl.col('관객수', '누적관객수', '스크린수', '상영횟수').str.replace_all(',', '').cast(pl.Int64)
)
df_boxoffice = df_boxoffice.with_columns(
    pl.col('장르').str.split(','),
    pl.col('배우').str.split(','),
)
df_boxoffice = (df_boxoffice.filter(
    pl.col('순위').is_not_null()).
                filter(~(pl.col('장르').list.contains('성인물(에로)'))).
                filter(pl.col('매출액') != 0).
                filter(~((pl.col('관객수') <= 1) & (pl.col('누적관객수') <= 1) &
                         (pl.col('스크린수') <= 1) & (pl.col('상영횟수') <= 1)))
                )

7.1 데이터프레임 조인

In [4]:
df_seat_rate = pl.read_csv("./movie/KOBIS_기간별_좌석점유율.csv",
                           skip_rows = 4, try_parse_dates=True)
df_seat_rate

순위,영화명,개봉일,좌석판매율,좌석점유율,좌석수,매출액,누적매출액,관객수,누적관객수
i64,str,date,str,str,str,str,str,str,str
1,"""파묘""",2024-02-22,"""22.2%""","""6.1%""","""53,450,628""","""115,157,490,414""","""115,157,490,414""","""11,914,784""","""11,914,784"""
2,"""범죄도시4""",2024-04-24,"""21.5%""","""6.0%""","""53,350,464""","""110,050,000,734""","""110,050,000,734""","""11,502,779""","""11,502,779"""
3,"""인사이드 아웃 2""",2024-06-12,"""17.5%""","""5.7%""","""50,018,740""","""84,509,284,036""","""84,509,284,036""","""8,799,013""","""8,799,013"""
4,"""베테랑2""",2024-09-13,"""18.3%""","""4.6%""","""40,990,441""","""72,474,035,814""","""72,474,035,814""","""7,525,339""","""7,525,339"""
5,"""파일럿""",2024-07-31,"""15.7%""","""3.4%""","""30,046,075""","""43,204,079,201""","""43,204,079,201""","""4,718,036""","""4,718,036"""
…,…,…,…,…,…,…,…,…,…
1438,"""파이란""",2001-04-28,"""0.0%""","""0.0%""","""424""","""0""","""13,402,270""","""0""","""1,380"""
1438,"""퍼펙트 게임""",2011-12-21,"""0.0%""","""0.0%""","""212""","""0""","""11,220,412,500""","""0""","""1,507,085"""
1438,"""해가 서쪽에서 뜬다면""",1998-12-19,"""0.0%""","""0.0%""","""212""","""0""","""0""","""0""","""0"""


In [5]:
df_seat_rate = df_seat_rate.with_columns(
    pl.col('좌석수').str.replace_all(',', '').cast(pl.Int64),
    pl.col('좌석판매율', '좌석점유율').str.replace_all('%', '').cast(pl.Float64)
).select(pl.exclude(['매출액', '누적매출액', '관객수', '누적관객수']))
df_seat_rate

순위,영화명,개봉일,좌석판매율,좌석점유율,좌석수
i64,str,date,f64,f64,i64
1,"""파묘""",2024-02-22,22.2,6.1,53450628
2,"""범죄도시4""",2024-04-24,21.5,6.0,53350464
3,"""인사이드 아웃 2""",2024-06-12,17.5,5.7,50018740
4,"""베테랑2""",2024-09-13,18.3,4.6,40990441
5,"""파일럿""",2024-07-31,15.7,3.4,30046075
…,…,…,…,…,…
1438,"""파이란""",2001-04-28,0.0,0.0,424
1438,"""퍼펙트 게임""",2011-12-21,0.0,0.0,212
1438,"""해가 서쪽에서 뜬다면""",1998-12-19,0.0,0.0,212


In [6]:
(df_boxoffice.join(df_seat_rate, on = '영화명', how = 'inner', suffix = '_좌석').
 select(pl.exclude(['개봉일']))
)

순위,영화명,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,순위_좌석,개봉일_좌석,좌석판매율,좌석점유율,좌석수
i64,str,i64,str,i64,i64,i64,i64,i64,str,str,str,str,str,list[str],str,list[str],i64,date,f64,f64,i64
1,"""파묘""",115157490414,"""9.6%""",115157490414,11914784,11914784,2367,364955,"""한국""","""한국""","""(주)쇼박스""","""(주)쇼박스""","""15세이상관람가""","[""미스터리"", ""공포(호러)""]","""장재현""","[""최민식"", ""김고은"", … ""김하영""]",1,2024-02-22,22.2,6.1,53450628
2,"""범죄도시4""",110050000734,"""9.2%""",110050000734,11502779,11502779,2980,375434,"""한국""","""한국""","""주식회사 빅펀치픽쳐스,(주)홍필름,(주)비에이엔터테인먼…","""주식회사 에이비오엔터테인먼트,플러스엠 엔터테인먼트""","""15세이상관람가""","[""액션"", ""범죄""]","""허명행""","[""김무열"", ""박지환"", … ""조석인""]",2,2024-04-24,21.5,6.0,53350464
3,"""인사이드 아웃 2""",84509284036,"""7.1%""",84509284036,8799013,8799013,2619,331341,"""미국""","""미국""","""월트 디즈니 픽쳐스,픽사 애니메이션 스튜디오""","""월트디즈니컴퍼니코리아 유한책임회사""","""전체관람가""","[""애니메이션""]","""켈시 만""","[""에이미 포엘러"", ""토니 헤일""]",3,2024-06-12,17.5,5.7,50018740
4,"""베테랑2""",72474035814,"""6.1%""",72474035814,7525339,7525339,2691,279673,"""한국""","""한국""","""(주)외유내강""","""(주)씨제이이엔엠""","""15세이상관람가""","[""액션"", ""범죄""]","""류승완""","[""황정민"", ""정해인"", … ""이상협""]",4,2024-09-13,18.3,4.6,40990441
5,"""파일럿""",43204079201,"""3.6%""",43204079201,4718036,4718036,1956,203283,"""한국""","""한국""","""쇼트케이크(주),(주)무비락""","""롯데컬처웍스(주)롯데엔터테인먼트""","""12세이상관람가""","[""코미디""]","""김한결""","[""조정석"", ""이주명"", … ""한지혁""]",5,2024-07-31,15.7,3.4,30046075
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3345,"""귀멸의 칼날: 주합회의·나비저택 편""",14000,"""0.0%""",263633000,2,27688,1,1,"""일본""","""일본""",null,"""(주)박수엔터테인먼트""","""15세이상관람가""","[""애니메이션""]","""소토자키 하루오""","[""하나에 나츠키"", ""키토 아카리"", … ""마츠오카 요시츠구""]",703,2021-12-01,1.5,0.0,130
3365,"""귀멸의 칼날: 남매의 연""",7000,"""0.0%""",645556440,1,65537,1,1,"""일본""","""일본""",null,"""(주)박수엔터테인먼트""","""15세이상관람가""","[""애니메이션""]","""소토자키 하루오""","[""하나에 나츠키"", ""키토 아카리"", … ""마츠오카 요시츠구""]",706,2021-11-10,0.7,0.0,130
3365,"""귀멸의 칼날: 아사쿠사 편""",7000,"""0.0%""",231539500,1,22389,1,1,"""일본""","""일본""",null,"""(주)박수엔터테인먼트""","""15세이상관람가""","[""애니메이션""]","""소토자키 하루오""",null,706,2022-08-25,0.7,0.0,130


In [7]:
(df_boxoffice.join(df_seat_rate, on = '영화명', how = 'left', suffix = '_좌석').
 select(pl.exclude(['개봉일'])).tail()
)

순위,영화명,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,순위_좌석,개봉일_좌석,좌석판매율,좌석점유율,좌석수
i64,str,i64,str,i64,i64,i64,i64,i64,str,str,str,str,str,list[str],str,list[str],i64,date,f64,f64,i64
3365,"""평평남녀""",8000,"""0.0%""",11582200,1,1317,1,1,"""한국""","""한국""","""브릿지 프로덕션,냉이영화""","""영화배급협동조합 씨네소파""","""12세이상관람가""","[""드라마"", ""멜로/로맨스""]","""김수정""","[""이태경"", ""이한주"", … ""박종환""]",null,null,null,null,null
3365,"""한 남자""",10000,"""0.0%""",139270250,1,15939,1,1,"""일본""","""일본""",null,"""트윈플러스파트너스(주)""","""12세이상관람가""","[""범죄"", ""멜로/로맨스"", ""스릴러""]","""이시카와 케이""","[""츠마부키 사토시"", ""안도 사쿠라""]",null,null,null,null,null
3365,"""헝거""",3000,"""0.0%""",6007060,1,778,1,1,"""한국""","""한국""","""영화사 케디필름""","""(주)디오시네마,(주)태양미디어그룹""","""12세이상관람가""","[""SF"", ""드라마""]","""강다연""","[""김유나"", ""최윤우"", … ""조은강""]",null,null,null,null,null
3365,"""호피와 차돌바위""",8000,"""0.0%""",28000,1,4,1,1,"""한국""","""한국""",null,null,null,"[""애니메이션""]","""신동헌""",null,null,null,null,null,null
3365,"""희수""",5000,"""0.0%""",4795500,1,617,1,1,"""한국""","""한국""","""영화사 정원""","""영화사 정원""","""12세이상관람가""","[""드라마""]","""감정원""","[""공민정""]",null,null,null,null,null


In [8]:
(df_boxoffice.join(df_seat_rate, on = '영화명', how = 'right', suffix = '_좌석').
 select(pl.exclude(['개봉일'])).tail()
)

순위,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,순위_좌석,영화명,개봉일_좌석,좌석판매율,좌석점유율,좌석수
i64,i64,str,i64,i64,i64,i64,i64,str,str,str,str,str,list[str],str,list[str],i64,str,date,f64,f64,i64
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1438,"""파이란""",2001-04-28,0.0,0.0,424
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1438,"""퍼펙트 게임""",2011-12-21,0.0,0.0,212
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1438,"""해가 서쪽에서 뜬다면""",1998-12-19,0.0,0.0,212
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1438,"""헌트""",2022-08-10,0.0,0.0,142
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1438,"""황산벌""",2003-10-17,0.0,0.0,424


In [9]:
(df_boxoffice.join(df_seat_rate, on = '영화명', how = 'full', suffix = '_좌석').
 select(pl.exclude(['개봉일']))
)

순위,영화명,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,순위_좌석,영화명_좌석,개봉일_좌석,좌석판매율,좌석점유율,좌석수
i64,str,i64,str,i64,i64,i64,i64,i64,str,str,str,str,str,list[str],str,list[str],i64,str,date,f64,f64,i64
1,"""파묘""",115157490414,"""9.6%""",115157490414,11914784,11914784,2367,364955,"""한국""","""한국""","""(주)쇼박스""","""(주)쇼박스""","""15세이상관람가""","[""미스터리"", ""공포(호러)""]","""장재현""","[""최민식"", ""김고은"", … ""김하영""]",1,"""파묘""",2024-02-22,22.2,6.1,53450628
2,"""범죄도시4""",110050000734,"""9.2%""",110050000734,11502779,11502779,2980,375434,"""한국""","""한국""","""주식회사 빅펀치픽쳐스,(주)홍필름,(주)비에이엔터테인먼…","""주식회사 에이비오엔터테인먼트,플러스엠 엔터테인먼트""","""15세이상관람가""","[""액션"", ""범죄""]","""허명행""","[""김무열"", ""박지환"", … ""조석인""]",2,"""범죄도시4""",2024-04-24,21.5,6.0,53350464
3,"""인사이드 아웃 2""",84509284036,"""7.1%""",84509284036,8799013,8799013,2619,331341,"""미국""","""미국""","""월트 디즈니 픽쳐스,픽사 애니메이션 스튜디오""","""월트디즈니컴퍼니코리아 유한책임회사""","""전체관람가""","[""애니메이션""]","""켈시 만""","[""에이미 포엘러"", ""토니 헤일""]",3,"""인사이드 아웃 2""",2024-06-12,17.5,5.7,50018740
4,"""베테랑2""",72474035814,"""6.1%""",72474035814,7525339,7525339,2691,279673,"""한국""","""한국""","""(주)외유내강""","""(주)씨제이이엔엠""","""15세이상관람가""","[""액션"", ""범죄""]","""류승완""","[""황정민"", ""정해인"", … ""이상협""]",4,"""베테랑2""",2024-09-13,18.3,4.6,40990441
5,"""파일럿""",43204079201,"""3.6%""",43204079201,4718036,4718036,1956,203283,"""한국""","""한국""","""쇼트케이크(주),(주)무비락""","""롯데컬처웍스(주)롯데엔터테인먼트""","""12세이상관람가""","[""코미디""]","""김한결""","[""조정석"", ""이주명"", … ""한지혁""]",5,"""파일럿""",2024-07-31,15.7,3.4,30046075
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,706,"""집주인 버섯 맛에 길들여진 가사도우미""",2024-02-03,1.2,0.0,83
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,706,"""돌아온 형수의 순수 욕정""",2024-12-05,0.6,0.0,146
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,564,"""여직원과 야근하다 한 떡""",2024-01-31,21.9,0.0,91


In [10]:
df_boxoffice.join(df_seat_rate, on = '영화명', how = 'semi', suffix = '_좌석')

순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우
i64,str,date,i64,str,i64,i64,i64,i64,i64,str,str,str,str,str,list[str],str,list[str]
1,"""파묘""",2024-02-22,115157490414,"""9.6%""",115157490414,11914784,11914784,2367,364955,"""한국""","""한국""","""(주)쇼박스""","""(주)쇼박스""","""15세이상관람가""","[""미스터리"", ""공포(호러)""]","""장재현""","[""최민식"", ""김고은"", … ""김하영""]"
2,"""범죄도시4""",2024-04-24,110050000734,"""9.2%""",110050000734,11502779,11502779,2980,375434,"""한국""","""한국""","""주식회사 빅펀치픽쳐스,(주)홍필름,(주)비에이엔터테인먼…","""주식회사 에이비오엔터테인먼트,플러스엠 엔터테인먼트""","""15세이상관람가""","[""액션"", ""범죄""]","""허명행""","[""김무열"", ""박지환"", … ""조석인""]"
3,"""인사이드 아웃 2""",2024-06-12,84509284036,"""7.1%""",84509284036,8799013,8799013,2619,331341,"""미국""","""미국""","""월트 디즈니 픽쳐스,픽사 애니메이션 스튜디오""","""월트디즈니컴퍼니코리아 유한책임회사""","""전체관람가""","[""애니메이션""]","""켈시 만""","[""에이미 포엘러"", ""토니 헤일""]"
4,"""베테랑2""",2024-09-13,72474035814,"""6.1%""",72474035814,7525339,7525339,2691,279673,"""한국""","""한국""","""(주)외유내강""","""(주)씨제이이엔엠""","""15세이상관람가""","[""액션"", ""범죄""]","""류승완""","[""황정민"", ""정해인"", … ""이상협""]"
5,"""파일럿""",2024-07-31,43204079201,"""3.6%""",43204079201,4718036,4718036,1956,203283,"""한국""","""한국""","""쇼트케이크(주),(주)무비락""","""롯데컬처웍스(주)롯데엔터테인먼트""","""12세이상관람가""","[""코미디""]","""김한결""","[""조정석"", ""이주명"", … ""한지혁""]"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3345,"""귀멸의 칼날: 주합회의·나비저택 편""",2021-12-01,14000,"""0.0%""",263633000,2,27688,1,1,"""일본""","""일본""",null,"""(주)박수엔터테인먼트""","""15세이상관람가""","[""애니메이션""]","""소토자키 하루오""","[""하나에 나츠키"", ""키토 아카리"", … ""마츠오카 요시츠구""]"
3365,"""귀멸의 칼날: 남매의 연""",2021-11-10,7000,"""0.0%""",645556440,1,65537,1,1,"""일본""","""일본""",null,"""(주)박수엔터테인먼트""","""15세이상관람가""","[""애니메이션""]","""소토자키 하루오""","[""하나에 나츠키"", ""키토 아카리"", … ""마츠오카 요시츠구""]"
3365,"""귀멸의 칼날: 아사쿠사 편""",2022-08-25,7000,"""0.0%""",231539500,1,22389,1,1,"""일본""","""일본""",null,"""(주)박수엔터테인먼트""","""15세이상관람가""","[""애니메이션""]","""소토자키 하루오""",null


In [11]:
df_boxoffice.join(df_seat_rate, on = '영화명', how = 'anti', suffix = '_좌석')

순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우
i64,str,date,i64,str,i64,i64,i64,i64,i64,str,str,str,str,str,list[str],str,list[str]
27,"""건국전쟁""",2024-02-01,10906368514,"""0.9%""",10906368514,1173927,1173927,956,49180,"""한국""","""한국""","""다큐스토리""","""다큐스토리""","""12세이상관람가""","[""다큐멘터리""]","""김덕영""",null
52,"""남은 인생 10년""",2023-05-24,4252407808,"""0.4%""",5721326408,425196,562828,529,25826,"""일본""","""일본""",null,"""(주)바이포엠스튜디오,(주)디스테이션""","""12세이상관람가""","[""멜로/로맨스""]","""후지이 미치히토""","[""고마츠 나나"", ""사카구치 켄타로"", … ""릴리 프랭키""]"
56,"""소풍""",2024-02-07,3207715943,"""0.3%""",3207715943,363932,363932,729,24674,"""한국""","""한국""","""(주)로케트필름""","""롯데컬처웍스(주)롯데엔터테인먼트""","""12세이상관람가""","[""드라마""]","""김용균""","[""나문희"", ""김영옥"", … ""성병숙""]"
69,"""비긴 어게인""",2014-08-13,2125870640,"""0.2%""",29573469671,231811,3712315,475,16506,"""미국""","""미국""",null,"""판씨네마(주)""","""15세이상관람가""","[""멜로/로맨스""]","""존 카니""","[""키이라 나이틀리"", ""마크 러팔로"", … ""캐서린 키너""]"
71,"""소년시절의 너""",2020-07-09,2027736150,"""0.2%""",2704847980,212185,292798,467,14450,"""중국""","""중국""",null,"""(주)바이포엠스튜디오,(주)영화특별시에스엠씨,(주)콘텐…","""15세이상관람가""","[""멜로/로맨스"", ""드라마"", … ""가족""]","""증국상""","[""주동우""]"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3365,"""평평남녀""",2022-04-28,8000,"""0.0%""",11582200,1,1317,1,1,"""한국""","""한국""","""브릿지 프로덕션,냉이영화""","""영화배급협동조합 씨네소파""","""12세이상관람가""","[""드라마"", ""멜로/로맨스""]","""김수정""","[""이태경"", ""이한주"", … ""박종환""]"
3365,"""한 남자""",2023-08-30,10000,"""0.0%""",139270250,1,15939,1,1,"""일본""","""일본""",null,"""트윈플러스파트너스(주)""","""12세이상관람가""","[""범죄"", ""멜로/로맨스"", ""스릴러""]","""이시카와 케이""","[""츠마부키 사토시"", ""안도 사쿠라""]"
3365,"""헝거""",2022-04-21,3000,"""0.0%""",6007060,1,778,1,1,"""한국""","""한국""","""영화사 케디필름""","""(주)디오시네마,(주)태양미디어그룹""","""12세이상관람가""","[""SF"", ""드라마""]","""강다연""","[""김유나"", ""최윤우"", … ""조은강""]"


In [12]:
df_boxoffice.join(df_seat_rate, how = 'cross', suffix = '_좌석')

순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,순위_좌석,영화명_좌석,개봉일_좌석,좌석판매율,좌석점유율,좌석수
i64,str,date,i64,str,i64,i64,i64,i64,i64,str,str,str,str,str,list[str],str,list[str],i64,str,date,f64,f64,i64
1,"""파묘""",2024-02-22,115157490414,"""9.6%""",115157490414,11914784,11914784,2367,364955,"""한국""","""한국""","""(주)쇼박스""","""(주)쇼박스""","""15세이상관람가""","[""미스터리"", ""공포(호러)""]","""장재현""","[""최민식"", ""김고은"", … ""김하영""]",1,"""파묘""",2024-02-22,22.2,6.1,53450628
1,"""파묘""",2024-02-22,115157490414,"""9.6%""",115157490414,11914784,11914784,2367,364955,"""한국""","""한국""","""(주)쇼박스""","""(주)쇼박스""","""15세이상관람가""","[""미스터리"", ""공포(호러)""]","""장재현""","[""최민식"", ""김고은"", … ""김하영""]",2,"""범죄도시4""",2024-04-24,21.5,6.0,53350464
1,"""파묘""",2024-02-22,115157490414,"""9.6%""",115157490414,11914784,11914784,2367,364955,"""한국""","""한국""","""(주)쇼박스""","""(주)쇼박스""","""15세이상관람가""","[""미스터리"", ""공포(호러)""]","""장재현""","[""최민식"", ""김고은"", … ""김하영""]",3,"""인사이드 아웃 2""",2024-06-12,17.5,5.7,50018740
1,"""파묘""",2024-02-22,115157490414,"""9.6%""",115157490414,11914784,11914784,2367,364955,"""한국""","""한국""","""(주)쇼박스""","""(주)쇼박스""","""15세이상관람가""","[""미스터리"", ""공포(호러)""]","""장재현""","[""최민식"", ""김고은"", … ""김하영""]",4,"""베테랑2""",2024-09-13,18.3,4.6,40990441
1,"""파묘""",2024-02-22,115157490414,"""9.6%""",115157490414,11914784,11914784,2367,364955,"""한국""","""한국""","""(주)쇼박스""","""(주)쇼박스""","""15세이상관람가""","[""미스터리"", ""공포(호러)""]","""장재현""","[""최민식"", ""김고은"", … ""김하영""]",5,"""파일럿""",2024-07-31,15.7,3.4,30046075
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3365,"""희수""",2022-12-15,5000,"""0.0%""",4795500,1,617,1,1,"""한국""","""한국""","""영화사 정원""","""영화사 정원""","""12세이상관람가""","[""드라마""]","""감정원""","[""공민정""]",1438,"""파이란""",2001-04-28,0.0,0.0,424
3365,"""희수""",2022-12-15,5000,"""0.0%""",4795500,1,617,1,1,"""한국""","""한국""","""영화사 정원""","""영화사 정원""","""12세이상관람가""","[""드라마""]","""감정원""","[""공민정""]",1438,"""퍼펙트 게임""",2011-12-21,0.0,0.0,212
3365,"""희수""",2022-12-15,5000,"""0.0%""",4795500,1,617,1,1,"""한국""","""한국""","""영화사 정원""","""영화사 정원""","""12세이상관람가""","[""드라마""]","""감정원""","[""공민정""]",1438,"""해가 서쪽에서 뜬다면""",1998-12-19,0.0,0.0,212


In [13]:
df_boxoffice.join_where(df_seat_rate, pl.col('순위') < pl.col('순위_좌석'), suffix = '_좌석')

순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,순위_좌석,영화명_좌석,개봉일_좌석,좌석판매율,좌석점유율,좌석수
i64,str,date,i64,str,i64,i64,i64,i64,i64,str,str,str,str,str,list[str],str,list[str],i64,str,date,f64,f64,i64
1,"""파묘""",2024-02-22,115157490414,"""9.6%""",115157490414,11914784,11914784,2367,364955,"""한국""","""한국""","""(주)쇼박스""","""(주)쇼박스""","""15세이상관람가""","[""미스터리"", ""공포(호러)""]","""장재현""","[""최민식"", ""김고은"", … ""김하영""]",2,"""범죄도시4""",2024-04-24,21.5,6.0,53350464
1,"""파묘""",2024-02-22,115157490414,"""9.6%""",115157490414,11914784,11914784,2367,364955,"""한국""","""한국""","""(주)쇼박스""","""(주)쇼박스""","""15세이상관람가""","[""미스터리"", ""공포(호러)""]","""장재현""","[""최민식"", ""김고은"", … ""김하영""]",3,"""인사이드 아웃 2""",2024-06-12,17.5,5.7,50018740
1,"""파묘""",2024-02-22,115157490414,"""9.6%""",115157490414,11914784,11914784,2367,364955,"""한국""","""한국""","""(주)쇼박스""","""(주)쇼박스""","""15세이상관람가""","[""미스터리"", ""공포(호러)""]","""장재현""","[""최민식"", ""김고은"", … ""김하영""]",4,"""베테랑2""",2024-09-13,18.3,4.6,40990441
1,"""파묘""",2024-02-22,115157490414,"""9.6%""",115157490414,11914784,11914784,2367,364955,"""한국""","""한국""","""(주)쇼박스""","""(주)쇼박스""","""15세이상관람가""","[""미스터리"", ""공포(호러)""]","""장재현""","[""최민식"", ""김고은"", … ""김하영""]",5,"""파일럿""",2024-07-31,15.7,3.4,30046075
1,"""파묘""",2024-02-22,115157490414,"""9.6%""",115157490414,11914784,11914784,2367,364955,"""한국""","""한국""","""(주)쇼박스""","""(주)쇼박스""","""15세이상관람가""","[""미스터리"", ""공포(호러)""]","""장재현""","[""최민식"", ""김고은"", … ""김하영""]",6,"""웡카""",2024-01-31,15.2,2.6,23091829
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1433,"""퍼펙트슛""",null,910000,"""0.0%""",910000,182,182,1,2,"""한국""","""한국""",null,null,null,"[""기타""]",null,null,1438,"""파이란""",2001-04-28,0.0,0.0,424
1433,"""퍼펙트슛""",null,910000,"""0.0%""",910000,182,182,1,2,"""한국""","""한국""",null,null,null,"[""기타""]",null,null,1438,"""퍼펙트 게임""",2011-12-21,0.0,0.0,212
1433,"""퍼펙트슛""",null,910000,"""0.0%""",910000,182,182,1,2,"""한국""","""한국""",null,null,null,"[""기타""]",null,null,1438,"""해가 서쪽에서 뜬다면""",1998-12-19,0.0,0.0,212


In [14]:
pl.concat([df_boxoffice.head(5), df_boxoffice.tail(5)], how = 'vertical')

순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우
i64,str,date,i64,str,i64,i64,i64,i64,i64,str,str,str,str,str,list[str],str,list[str]
1,"""파묘""",2024-02-22,115157490414,"""9.6%""",115157490414,11914784,11914784,2367,364955,"""한국""","""한국""","""(주)쇼박스""","""(주)쇼박스""","""15세이상관람가""","[""미스터리"", ""공포(호러)""]","""장재현""","[""최민식"", ""김고은"", … ""김하영""]"
2,"""범죄도시4""",2024-04-24,110050000734,"""9.2%""",110050000734,11502779,11502779,2980,375434,"""한국""","""한국""","""주식회사 빅펀치픽쳐스,(주)홍필름,(주)비에이엔터테인먼…","""주식회사 에이비오엔터테인먼트,플러스엠 엔터테인먼트""","""15세이상관람가""","[""액션"", ""범죄""]","""허명행""","[""김무열"", ""박지환"", … ""조석인""]"
3,"""인사이드 아웃 2""",2024-06-12,84509284036,"""7.1%""",84509284036,8799013,8799013,2619,331341,"""미국""","""미국""","""월트 디즈니 픽쳐스,픽사 애니메이션 스튜디오""","""월트디즈니컴퍼니코리아 유한책임회사""","""전체관람가""","[""애니메이션""]","""켈시 만""","[""에이미 포엘러"", ""토니 헤일""]"
4,"""베테랑2""",2024-09-13,72474035814,"""6.1%""",72474035814,7525339,7525339,2691,279673,"""한국""","""한국""","""(주)외유내강""","""(주)씨제이이엔엠""","""15세이상관람가""","[""액션"", ""범죄""]","""류승완""","[""황정민"", ""정해인"", … ""이상협""]"
5,"""파일럿""",2024-07-31,43204079201,"""3.6%""",43204079201,4718036,4718036,1956,203283,"""한국""","""한국""","""쇼트케이크(주),(주)무비락""","""롯데컬처웍스(주)롯데엔터테인먼트""","""12세이상관람가""","[""코미디""]","""김한결""","[""조정석"", ""이주명"", … ""한지혁""]"
3365,"""평평남녀""",2022-04-28,8000,"""0.0%""",11582200,1,1317,1,1,"""한국""","""한국""","""브릿지 프로덕션,냉이영화""","""영화배급협동조합 씨네소파""","""12세이상관람가""","[""드라마"", ""멜로/로맨스""]","""김수정""","[""이태경"", ""이한주"", … ""박종환""]"
3365,"""한 남자""",2023-08-30,10000,"""0.0%""",139270250,1,15939,1,1,"""일본""","""일본""",null,"""트윈플러스파트너스(주)""","""12세이상관람가""","[""범죄"", ""멜로/로맨스"", ""스릴러""]","""이시카와 케이""","[""츠마부키 사토시"", ""안도 사쿠라""]"
3365,"""헝거""",2022-04-21,3000,"""0.0%""",6007060,1,778,1,1,"""한국""","""한국""","""영화사 케디필름""","""(주)디오시네마,(주)태양미디어그룹""","""12세이상관람가""","[""SF"", ""드라마""]","""강다연""","[""김유나"", ""최윤우"", … ""조은강""]"
3365,"""호피와 차돌바위""",null,8000,"""0.0%""",28000,1,4,1,1,"""한국""","""한국""",null,null,null,"[""애니메이션""]","""신동헌""",null


In [15]:
pl.concat([df_boxoffice.select(['순위', '영화명', '개봉일']).tail(5),
           df_seat_rate.select([pl.col('영화명').alias('영화명_좌석'),
                                '좌석판매율', '좌석점유율', '좌석수']).head(3)],
          how = 'horizontal')


순위,영화명,개봉일,영화명_좌석,좌석판매율,좌석점유율,좌석수
i64,str,date,str,f64,f64,i64
3365,"""평평남녀""",2022-04-28,"""파묘""",22.2,6.1,53450628
3365,"""한 남자""",2023-08-30,"""범죄도시4""",21.5,6.0,53350464
3365,"""헝거""",2022-04-21,"""인사이드 아웃 2""",17.5,5.7,50018740
3365,"""호피와 차돌바위""",null,null,null,null,null
3365,"""희수""",2022-12-15,null,null,null,null


In [16]:
pl.concat([df_boxoffice.head(5), df_seat_rate.head(5)], how = 'diagonal')

순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,좌석판매율,좌석점유율,좌석수
i64,str,date,i64,str,i64,i64,i64,i64,i64,str,str,str,str,str,list[str],str,list[str],f64,f64,i64
1,"""파묘""",2024-02-22,115157490414,"""9.6%""",115157490414,11914784,11914784,2367,364955,"""한국""","""한국""","""(주)쇼박스""","""(주)쇼박스""","""15세이상관람가""","[""미스터리"", ""공포(호러)""]","""장재현""","[""최민식"", ""김고은"", … ""김하영""]",null,null,null
2,"""범죄도시4""",2024-04-24,110050000734,"""9.2%""",110050000734,11502779,11502779,2980,375434,"""한국""","""한국""","""주식회사 빅펀치픽쳐스,(주)홍필름,(주)비에이엔터테인먼…","""주식회사 에이비오엔터테인먼트,플러스엠 엔터테인먼트""","""15세이상관람가""","[""액션"", ""범죄""]","""허명행""","[""김무열"", ""박지환"", … ""조석인""]",null,null,null
3,"""인사이드 아웃 2""",2024-06-12,84509284036,"""7.1%""",84509284036,8799013,8799013,2619,331341,"""미국""","""미국""","""월트 디즈니 픽쳐스,픽사 애니메이션 스튜디오""","""월트디즈니컴퍼니코리아 유한책임회사""","""전체관람가""","[""애니메이션""]","""켈시 만""","[""에이미 포엘러"", ""토니 헤일""]",null,null,null
4,"""베테랑2""",2024-09-13,72474035814,"""6.1%""",72474035814,7525339,7525339,2691,279673,"""한국""","""한국""","""(주)외유내강""","""(주)씨제이이엔엠""","""15세이상관람가""","[""액션"", ""범죄""]","""류승완""","[""황정민"", ""정해인"", … ""이상협""]",null,null,null
5,"""파일럿""",2024-07-31,43204079201,"""3.6%""",43204079201,4718036,4718036,1956,203283,"""한국""","""한국""","""쇼트케이크(주),(주)무비락""","""롯데컬처웍스(주)롯데엔터테인먼트""","""12세이상관람가""","[""코미디""]","""김한결""","[""조정석"", ""이주명"", … ""한지혁""]",null,null,null
1,"""파묘""",2024-02-22,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,22.2,6.1,53450628
2,"""범죄도시4""",2024-04-24,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,21.5,6.0,53350464
3,"""인사이드 아웃 2""",2024-06-12,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,17.5,5.7,50018740
4,"""베테랑2""",2024-09-13,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,18.3,4.6,40990441


In [17]:
df_boxoffice.select(['영화명', '등급', '대표국적', '관객수', '매출액']).filter(
    pl.col('대표국적').is_in(['한국', '미국', '일본', '프랑스', '기타', '영국',
                             '이탈리아', '독일' '중국', '캐나다'])
).pivot('대표국적', index = '등급', values = '관객수', aggregate_function = 'sum')

등급,한국,미국,일본,영국,프랑스,캐나다,이탈리아,기타
str,i64,i64,i64,i64,i64,i64,i64,i64
"""15세이상관람가""",43891467,8035572,819819,49881,201424,5122,10167,2954
"""전체관람가""",4182644,25509345,2838453,165202,65871,24104,17387,1714
"""12세이상관람가""",22229389,5281355,2765070,56613,80340,1900,32622,123
"""청소년관람불가""",1073348,3370813,47180,156215,29146,12606,11708,261
"""15세관람가,15세이상관람가""",null,193018,929,null,null,null,null,null
…,…,…,…,…,…,…,…,…
"""기타""",191,null,null,null,null,null,null,null
"""15세이상관람가,전체관람가""",null,null,null,null,183,null,null,null
"""18세관람가,15세이상관람가""",null,null,null,null,86,null,null,null


In [18]:
df_boxoffice.select(['영화명', '등급', '대표국적', '관객수', '매출액']).filter(
    pl.col('대표국적').is_in(['한국', '미국', '일본', '프랑스', '기타',
                             '영국', '이탈리아', '독일' '중국', '캐나다'])
).pivot('대표국적', index = '등급', values = '관객수', aggregate_function = 'sum').unpivot(
    on = ['한국', '미국', '일본', '영국', '프랑스', '캐나다',
          '이탈리아', '기타'], index = '등급', variable_name = '대표국적'
)


등급,대표국적,value
str,str,i64
"""15세이상관람가""","""한국""",43891467
"""전체관람가""","""한국""",4182644
"""12세이상관람가""","""한국""",22229389
"""청소년관람불가""","""한국""",1073348
"""15세관람가,15세이상관람가""","""한국""",null
…,…,…
"""기타""","""기타""",null
"""15세이상관람가,전체관람가""","""기타""",null
"""18세관람가,15세이상관람가""","""기타""",null
